### Part 1:

Basic recursion, took me awhile to figure this out but needed the following logic:
- If an int, then return it
- If not an int but a key, we want to append to a list:
    - Recursion kicks in here - we keep running until we can return an int, which is appended to the processing list
- If not an int and not a key then it must be an operand (or a horrible trick!):
    - At this point we can return the solution to a math equation using the potentially risky `eval`
    
    
#### A Bit More On Eval:
- https://realpython.com/python-eval-function/
- Major risk:
    - "eval() is considered insecure because it allows you (or your users) to dynamically execute arbitrary Python code."
    - The biggest risk looks to be if you pass inputs from a user with `eval`. For this problem I am not receiving any user inputs, just using `eval` off of a dictionary I am familiar with so not as concerned.
- Note: tried using `ast` but does not seem to support basic arithmetic so scrapped that.

In [1]:
from ast import literal_eval # couldn't get this working for the problem at hand
import numpy as np
import sys

def recursion(d, k, debugger = False):
    """Iterate over values in key, adding colors to a main list representing bags that can be
    stored in outer bag (k)
    """
    if debugger:
        print(k)
    
    processing = [] # storage of our ints

    # iterate over each value of list
    for c in d[k]:
        # when we have an int we add to processing, then return it (this worked?)
        if type(c) == int:
            return c
        
        # when we have a value that is a key we use recursion to get this to an intent,
        # this int is then added to processing
        if c in d.keys():
            processing.append(recursion(d, c, debugger))
            
        # anything else must be an operation (probably better to actually include our finite set of operations)
        else:
            operation = c
    if debugger:
        print(processing) 
        print(operation)
        
    # once we have iterated through we can solve our equation
    # this allows us to return recursively, meaning processing will keep being populated
    # from the innermost to outermost
    return eval(f"{processing[0]}{operation}{processing[1]}")

# Sample: 
with open("data/day21_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

# build an ops dict of either values or instructions for operations
ops_dict = {l.split(':')[0]: [x.strip() for x in l.split(': ')[1].split(' ')] for l in lines}

# convert all len 1 to int for easier processing
for k,v in ops_dict.items():
    if len(v) == 1:
        ops_dict[k] = [int(x) for x in v]

#print(ops_dict)

# Run w/ debugger: Final value is value for "root"
recursion(ops_dict, 'root', debugger = True)

root
pppw
cczh
sllz
lgvd
ljgn
ptdq
humn
dvpt
[5, 3]
-
[2, 2]
*
[4, 4]
+
lfqf
[8, 4]
/
sjmn
drzm
hmdt
zczc
[32, 2]
-
dbpl
[30, 5]
*
[2.0, 150]
+


152.0

In [2]:
# Run part 1, no debugger:
with open("data/day21.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

# build an ops dict of either values or instructions for operations
ops_dict = {l.split(':')[0]: [x.strip() for x in l.split(': ')[1].split(' ')] for l in lines}

# convert all len 1 to int for easier processing
for k,v in ops_dict.items():
    if len(v) == 1:
        ops_dict[k] = [int(x) for x in v]

# no debugger, root value
recursion(ops_dict, 'root')

41857219607906.0

### Part 2: 

I initially tried to solve this with brute force from `0`, pass in a new value for `humn` and once the boolean returned is `True` then success. 
- Pretty soon figured this wouldn't work, actually ran into errors on my real dat which I didn't understand so scrapped this approach thinking I was causing some math errors somewhere. 

#### Bug That Took Me Awhile To Catch:

I made the incorrect assumption that `root` in the sample data and my data connected to the same nodes, this is definitely not true and was causing a lot of indexing errors.

#### Hint for Part 2:

Wish I had spent a little more time before resorting to the reddit forum and looking for a hint. The hint automatically made sense - `If you find a humn value for which root says "too low". And another for which root says "too high".`

Therefore, we can use math to reduce the range of eligible values at each step, and choose a random one until we yield a solution.

Instead of using `==` I swapped to `-` for the root.
- An output < 0 indicates a low guess, so we increase the `min_guess`
- An output > 0 indicates a high guess, so we reduce the `max_guess`
- An output of 0 indicates a correct guess!
- My initial values were 0 - `sys.maxsize` ()

Another wrinkle of course:
- My actual input would get trapped in an infinite loop of the equal to `maxsize`.
- Initially this threw me off, and I just swapped the output logic (> 0 is a high guess now, < 0 is a low guess). This resolved my problem....so really stumbling into an answer here.

In [3]:
# Sample: 
with open("data/day21_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

# build an ops dict of either values or instructions for operations
ops_dict = {l.split(':')[0]: [x.strip() for x in l.split(': ')[1].split(' ')] for l in lines}

# convert all len 1 to int for easier processing
for k,v in ops_dict.items():
    if len(v) == 1:
        ops_dict[k] = [int(x) for x in v]

# Modify root to show us the diff? this can help us modify our guess
ops_dict['root'] = ['pppw', '-', 'sjmn']
ops_dict['humn'] = [151]
out = recursion(ops_dict, 'root', debugger = False)
out

-75.0

In [4]:
# iterate and rebuild boundaries
min_guess = 0
max_guess = sys.maxsize

out = False
steps = 0

while True:
    # take a guess between boundaries
    value_used = np.random.randint(min_guess, max_guess)
    print(value_used)
    
    
    # process
    steps += 1
    ops_dict['humn'] = [int(value_used)]
    out = recursion(ops_dict, 'root', debugger = False)
    
    # boundary search
    if out < 0:
        min_guess = max(value_used, min_guess)
    elif out > 0:
        max_guess = min(value_used, max_guess)   
    else:
        print(f"Took {steps} steps to find correct value: {value_used}")
        break
        
        
    if steps % 1_000 == 0:
        print(f"Taken {steps} steps and no solution :(")
        print(f"Next guess: {value_used}")

2347998988076804409
1405149182598363035
431617277102195337
108311534428620446
72170467163967059
46287636137340654
20764650610046676
1781405348638853
1674058264803961
1390012136991962
385940844056661
240453673375640
486583612976
50403320985
32819108120
31780085312
27161808589
1403349540
951917630
905405624
737516751
341240182
222251373
199662938
20765854
3933295
1073031
836038
366440
139035
116657
9773
5371
3132
1037
821
265
268
271
628
347
342
284
323
314
298
301
Took 47 steps to find correct value: 301


In [5]:
#
with open("data/day21.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

# build an ops dict of either values or instructions for operations
ops_dict = {l.split(':')[0]: [x.strip() for x in l.split(': ')[1].split(' ')] for l in lines}

# convert all len 1 to int for easier processing
for k,v in ops_dict.items():
    if len(v) == 1:
        ops_dict[k] = [int(x) for x in v]

# Modify root
ops_dict['root'] = ['fgtg', '-', 'pbtm']

min_v = 0
max_v = sys.maxsize

out = False
steps = 0

while True:
    # take a guess between boundaries
    value_used = np.random.randint(min_v, max_v)
    print(value_used)
    
    
    # process
    steps += 1
    ops_dict['humn'] = [int(value_used)]
    out = recursion(ops_dict, 'root', debugger = False)
    
    # boundary search
    if out > 0:
        min_v = max(value_used, min_v)
    elif out < 0:
        max_v = min(value_used, max_v)   
    else:
        print(f"Took {steps} steps to find correct value: {int(value_used)}")
        break
        
        
    if steps % 1_000 == 0:
        print(f"Taken {steps} steps and no solution :(")
        print(f"Next guess: {value_used}")

1455216391358372654
586650908453045824
94982939922494358
88744817505437108
77079038575936575
11804592945440648
2321221106796244
1897181872837496
1013244426768248
964693623252054
36236992033890
28117664406725
15089229314238
14780549780192
6883178239097
852623471906
3016851271490
6873177073901
3071805015229
6001821804627
4962332117379
4629322546241
4037067658807
3829512418864
3933221176801
3843050982527
3918102689370
3895821826618
3913280023176
3915465660336
3917387331946
3916790681394
3917221598381
3917085650773
3917022634530
3916947649249
3916931022847
3916942348010
3916939394890
3916931259968
3916934008698
3916939288768
3916934326065
3916935306997
3916935513732
3916937025072
3916935613375
3916936877606
3916936896879
3916936879478
3916936881589
3916936880515
3916936879559
3916936880341
3916936880405
3916936880489
3916936880451
3916936880439
3916936880442
3916936880443
3916936880450
3916936880444
3916936880449
3916936880447
3916936880448
Took 65 steps to find correct value: 391693688044